In [52]:
import sys

sys.path.append(r"/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master")
from Core_Definition import *
from Auto_Algorithm import *
from Visualization import *
import os
from SPAM import *
import numpy as np
import xlsxwriter as xlsx
from scipy import optimize
import random
import math
from math import floor
import uuid
import itertools

In [53]:
import json, pickle
import qiskit
#transpiler reducinig qiskit circuit to QASM strings
from qiskit import transpile
# Use AerSimulator
from qiskit_aer import AerSimulator
from qiskit import IBMQ, QuantumCircuit, execute
from qiskit.providers.ibmq.managed import IBMQJobManager
from qiskit.extensions import UnitaryGate
from qiskit.quantum_info import Pauli, Clifford, Statevector,Operator
from qiskit.visualization import array_to_latex
from scipy.stats import sem, unitary_group
from scipy.linalg import sqrtm,expm
import qiskit.quantum_info as qi
from qiskit.quantum_info import Pauli
from qiskit.providers.aer.noise import NoiseModel, pauli_error, amplitude_damping_error, ReadoutError
import pylatexenc

# functions

function for gates

In [54]:
#prepare eigenstate of each pauli
def prepare_pauli_eigenstate_gates(qc,pauli,q_index):
    if (pauli=="I"):
        pauli = random.choice(['X','Y','Z'])
    if (pauli=="X"):
        qc.Add_Gate(Quantum_Gate("RY",q_index,angle=2))
    elif (pauli=="Y"):
        qc.Add_Gate(Quantum_Gate("RX",q_index,angle=-2))
    elif (pauli=="Z"):
        pass

        
        
        
#should we set_mapping in this function? need to think about if it takes mapping into account   
def add_pauli_twirl(qc,n):
    pauliLayer = [random.choice(['I','X','Y','Z']) for j in range(n)]
    q_index=0
    for pauli in pauliLayer :
        if (pauli=="I"):
            pass
        elif (pauli=="X"):
            qc.Add_Gate(Quantum_Gate("RX",q_index,angle=1))
        elif (pauli=="Y"):
            qc.Add_Gate(Quantum_Gate("RY",q_index,angle=1))
        elif (pauli=="Z"):
            qc.Add_Gate(Quantum_Gate("AZ",q_index,angle=1))
        q_index+=1
    return pauliLayer
    
def add_clifford_layer(qc,n,clifford):
    if (clifford=="CNOT"):
        for i in range (floor(n/2)):
            qc.Add_Gate(Quantum_Gate("CNOT",2*i,2*i+1))
    if (clifford=="XX"):
        for i in range (floor(n/2)):
            qc.Add_Gate(Quantum_Gate("FTXA",2*i,2*i+1,angle=1/4))

def measure_pauli_1q_gates(qc,index,pauli=None):
    if pauli == 'I' or pauli == 'Z':
        pass
    elif pauli == 'X':
        qc.Add_Gate(Quantum_Gate("RY",index,angle=-2))
    elif pauli == 'Y':
        qc.Add_Gate(Quantum_Gate("RX",index,angle=2))
    else:
        assert 1==0            


def map_statepop_2_ibm_mapping(counts,n):
    counts_ibm_mapping=[0 for i in range (2**(2*n))]
    idx_ibm_mapping=[]
    counts_ibm_mapping_dic={}
    if len(str((2*n)))<2:
        f="00"+str(2*n)+"b"
    elif len(str((2*n)))<3:
        f="0"+str(2*n)+"b"
    else:
        raise ValueError("n is too big")
    for gates_idx in range (2**(2*n)):
        gates_idx_str=format(gates_idx,f)
        ibm_idx=0
        for i in range (2*n):
            ibm_idx+=2**(i)*int(gates_idx_str[i])
        ibm_idx_str=format(ibm_idx,f)
#         idx_ibm_mapping.append(ibm_idx_str)
        counts_ibm_mapping[ibm_idx]=counts[gates_idx]
        counts_ibm_mapping_dic[ibm_idx_str]=counts[gates_idx]
    return counts_ibm_mapping, counts_ibm_mapping_dic
        
def map_statepop_2_ibm_mapping_no_ancilla(counts,n):
    counts_ibm_mapping=[0 for i in range (2**(n))]
    idx_ibm_mapping=[]
    counts_ibm_mapping_dic={}
    if len(str(n))<2:
        f="00"+str(n)+"b"
    elif len(str(n))<3:
        f="0"+str(n)+"b"
    else:
        raise ValueError("n is too big")
    for gates_idx in range (2**(n)):
        gates_idx_str=format(gates_idx,f)
        ibm_idx=0
        for i in range (n):
            ibm_idx+=2**(i)*int(gates_idx_str[i])
        ibm_idx_str=format(ibm_idx,f)
#         idx_ibm_mapping.append(ibm_idx_str)
        counts_ibm_mapping[ibm_idx]=counts[gates_idx]
        counts_ibm_mapping_dic[ibm_idx_str]=counts[gates_idx]
    return counts_ibm_mapping, counts_ibm_mapping_dic

funtion for qiskit

In [55]:
#for qiskit

def prepare_pauli_eigenstate_1q(circuit,index,pauli=None):
	if pauli == 'I':
		pauli = random.choice(['X','Y','Z']) 
	# note: For Pauli I, prepare an arbitrary eigenstate.
	if pauli == 'Z':
		circuit.id([index])
	elif pauli == 'X':
		circuit.h([index])
	elif pauli == 'Y':
		circuit.h([index])
		circuit.s([index])
	else:
		print(pauli)
		assert 1==0

def pauli_gate_1q(circuit,index,pauli=None): #For stabilizer simulator to work, cannot use Pauli class
	if pauli == 'I':
		circuit.id([index])
	elif pauli == 'Z':
		circuit.z([index])
	elif pauli == 'X':
		circuit.x([index])
	elif pauli == 'Y':
		circuit.y([index])
	else:
		assert 1==0

def add_XX_clifford(circ,index):
    circ.z(2*index)
    circ.h(2*index)
    circ.x(2*index)
    circ.z(2*index)
    circ.cx(2*index,2*index+1)
    circ.s(2*index)
    circ.h(2*index)
    circ.x(2*index)
    circ.h(2*index+1)
    circ.s(2*index+1)
    circ.h(2*index+1)
        
def construct_measurement_qiskit(pauli_sample,qc_gates):
    pauliOp = Pauli(''.join(pauli_sample))
    pauliOp = pauliOp.evolve(Clifford(qc_gates).adjoint()) 
    measurement_setting = pauliOp.to_label()
# there could be a '-' in the Pauli label
    if measurement_setting[0].isupper() is False:
        measurement_setting = measurement_setting[1:]
    measurement_setting = measurement_setting[::-1]
    return measurement_setting
    #print(measurement_setting)

def measure_pauli_1q(circuit,index,pauli=None):
	if pauli == 'I' or pauli == 'Z':
		circuit.id([index])
	elif pauli == 'X':
		circuit.h([index])
	elif pauli == 'Y':
		circuit.s([index])
		circuit.s([index])
		circuit.s([index])
		circuit.h([index])
	else:
		assert 1==0

In [56]:
## Additional functions

def int_to_pauli(i,n):
    p = np.base_repr(i,base=4)
    p = '0'*(n-len(p)) + p
    p = p.replace('0','I').replace('1', 'X').replace('2', 'Y').replace('3', 'Z')
    return p

def commute(p,q):
    c = 1
    n = len(p)
    for i in range(n):
        if p[i] != 'I' and q[i] != 'I':
            if p[i] != q[i]:
                c *= -1
    return c

def fidelity_to_error(pauli_fidelity,n):
    N = 4**n
    pauli_error = {}
    for i in range(N):
        p = int_to_pauli(i,n)
        pauli_error[p] = 0
        for j in range(N):
            q = int_to_pauli(j,n)
            pauli_error[p] += pauli_fidelity[q] * commute(p,q) / N
    return pauli_error
    

In [59]:
# Set_Mapping([1,3,4,5,2])
Set_Mapping([2,3,5,4,1])
n=2
# ancillae=2
S=Quantum_Circuit(n,"qc")
S_explicit=Quantum_Circuit(n,"qc_e")

depth=[2**x for x in range(1,6)] #depth for Concatenation
C=20 #number of samples per depth
# all_circuits=[]#a list of lists. circuits for all depths. each element is a list of circuit of a certain depth d.
all_circuits_explicit={}
# circuits=[] #a list of circuits with a certain depth d
circuits_explicit=[]
# a list of circuits at all depth d for a certain pauli
circuits_explicit_pauli=[]
cb_data={} # the object that will be saved eventually
results={}
result_list=[]
clifford_layer="XX"

#for qiskit 
noise_model = NoiseModel()
eps=0.005
noise_model.add_all_qubit_quantum_error(pauli_error([('II',(1-eps)**2),('XX',eps**2),('IX',eps*(1-eps)),('XI',eps*(1-eps))]),['cx'])
# backend = AerSimulator()
backend = AerSimulator(method='stabilizer', noise_model=noise_model)
shots=2000

pauli_sample_list = [''.join(s) for s in itertools.product(['X','Y','Z'], repeat = n)]
for pauli_sample in pauli_sample_list:
    print ("\npauli label:",pauli_sample)
    all_circuits_explicit[pauli_sample]=[]
    cb_data[pauli_sample]={}
    cb_data[pauli_sample]["parameters"]={}
    cb_data[pauli_sample]["parameters"]['n'] = n 
    cb_data[pauli_sample]["parameters"]['shots'] = shots 
    cb_data[pauli_sample]["parameters"]['Lrange'] = depth
    cb_data[pauli_sample]["parameters"]['C'] = C
#     cb_data["parameters"]['repeat'] = repeat
    cb_data[pauli_sample]["result"]=[]
    for d in depth:
#     t_echo=d/2
        print ("\nDepth = %d"%d)
        print ("Number of samples = %d"%C)
        for i in range (C):
            #gates circuitt
#             S.depth=0
#             S.gates=[]
            #gates explicit cnot circuit    
            S_explicit.depth=0
            S_explicit.gates=[]
            #qiskit_circ=QuantumCircuit(2*nqubit,2*nqubit)
            qiskit_c_state = QuantumCircuit(n,n)
            qiskit_c_gates = QuantumCircuit(n)

            #gates circuit state prep
            for j in range(n):
                #gates CNOT block circuit
#                 prepare_pauli_eigenstate_gates(S,pauli_sample[n-j-1],j)
                #gates explicit CNOT circuit state prep
                prepare_pauli_eigenstate_gates(S_explicit,pauli_sample[n-j-1],j)
            #qiskit circuit state prep
            for j in range(n):
                prepare_pauli_eigenstate_1q(qiskit_c_state,j,pauli=pauli_sample[n-j-1])
            qiskit_c_state.barrier()


            for j in range(d):
                #gates circuit add pauli layer
#                 layer=add_pauli_twirl(S,n)
                #gates explicit cnot circuit add pauli layer
                layer=add_pauli_twirl(S_explicit,n)

                #qiskit circuit add pauli layer
                for q in range(n):
                    pauli_gate_1q(qiskit_c_gates,q,pauli=layer[q])


                #gates circuit add clifford layer
#                 add_clifford_layer(S,n,clifford_layer)
                #gates explicit cnot circuit add clifford layer
                add_clifford_layer(S_explicit,n,clifford_layer)

                #qiskit circuit add clifford layer
                if clifford_layer == 'Id':
                    pass
                elif clifford_layer == 'CNOT':
                    ngates = int(n/2)
                    for q in range(ngates):
                        qiskit_c_gates.cx(2*q,2*q+1)
                elif clifford_layer == 'XX':
                    ngates = int(n/2)
                    for q in range(ngates):
                        add_XX_clifford(qiskit_c_gates,q)
                qiskit_c_gates.barrier()
            
            #gates circuit add Pauli twirl
#             layer=add_pauli_twirl(S,n)
            #gates explicit cnot circuit add Pauli twirl
            layer=add_pauli_twirl(S_explicit,n)
            #qiskit circuit add Pauli twirl
            for j in range(n):
                pauli_gate_1q(qiskit_c_gates,j,pauli=layer[j])

            #save the stablizer representation
            cliffordOp = Clifford(qiskit_c_gates)
        
            #construct measurement basis
            measurement_basis=construct_measurement_qiskit(pauli_sample,qiskit_c_gates)
        
            #for qiskit: append gates to state
            qiskit_circ = qiskit_c_state.compose(qiskit_c_gates,range(n))

            #gates circuti measurement
#             for j in range (n):
#                 measure_pauli_1q_gates(S,j,pauli=measurement_basis[j])
            #gates explicit cnot circuti measurement
            for j in range (n):
                measure_pauli_1q_gates(S_explicit,j,pauli=measurement_basis[j])
            #qiskit measurement
            qiskit_circ.barrier()
            for j in range(n):
                measure_pauli_1q(qiskit_circ,j,pauli=measurement_basis[j])

            qiskit_circ.barrier()
            qiskit_circ.measure([j for j in range(n)], [j for j in range(n)])


            #for qiskit 
#             circ1.draw('mpl')
#             if (i==0):
# #                 qc_compiled = transpile(qiskit_circ, backend)
#                 print ("measurement basis:",measurement_basis)
#                 print (qiskit_circ)
#             qc_compiled = transpile(qiskit_circ, backend)
#             job_sim = backend.run(qc_compiled, shots=2000)
            job_sim = backend.run(qiskit_circ, shots=shots, max_parallel_experiments=0, memory = True) 
            result_sim = job_sim.result()
            counts_qiskit = result_sim.get_counts(qiskit_circ)

            #for gates
#             circuits.append(S.GatesLab_Sequence())
            circuits_explicit.append(S_explicit.GatesLab_Sequence())
            
#         print (S.GatesLab_Sequence())
#         print (S_explicit.GatesLab_Sequence())
#         statef=S.Simulate()
#         statef=S_explicit.Simulate()
#         statef.population=[round (x) for x in statef.population]
#         statef_explicit.population=[round (x) for x in statef_explicit.population]
#         print (statef.population)
#         print (statef_explicit.population)                     
        #package and format results
#             results["n"]=nqubit
            results["L"]=d
            results["circuit"] = qiskit_circ.qasm
            results["clifford"] = cliffordOp.to_dict()
            results["clifford_layer"] = clifford_layer
            results["pauli"] = measurement_basis
#         statef=S.Simulate()
            statef=S_explicit.Simulate()
            counts=[round(2000*x) for x in statef.population]
            counts_ibm_mapping, counts_ibm_mapping_dic=map_statepop_2_ibm_mapping_no_ancilla(counts,n)
            results["counts"]=counts_ibm_mapping_dic
#             results["counts"]=counts_qiskit
            if 0==0:
                for state in results["counts"]:
                    if results["counts"][state]!=0:
                        print (state, results["counts"][state])
                print(counts_qiskit)
#                 print (cliffordOp)
            cb_data[pauli_sample]["result"].append(results)
            results={}

            
        circuits_explicit_pauli.append(circuits_explicit)
#             all_circuits.append(circuits)
        
    #     for line in circuits_explicit:
    #         print (line)
#             circuits=[]
        circuits_explicit=[]
    all_circuits_explicit[pauli_sample]=circuits_explicit_pauli

pauli_sample_list = [''.join(s) for s in itertools.product(['X','Y','Z'], repeat = n)]
cb_data["n"] = n
cb_data["pauli_sample_list"] = pauli_sample_list    


pauli label: XX

Depth = 2
Number of samples = 20
10 2000
{'11': 16, '10': 1984}
01 2000
{'00': 22, '01': 1978}
01 2000
{'00': 18, '01': 1982}
10 2000
{'11': 26, '10': 1974}
10 2000
{'11': 21, '10': 1979}
01 2000
{'00': 13, '01': 1987}
00 2000
{'01': 28, '00': 1972}
10 2000
{'11': 21, '10': 1979}
00 2000
{'01': 13, '00': 1987}
10 2000
{'11': 25, '10': 1975}
00 2000
{'01': 25, '00': 1975}
01 2000
{'00': 20, '01': 1980}
01 2000
{'00': 17, '01': 1983}
10 2000
{'11': 19, '10': 1981}
11 2000
{'10': 30, '11': 1970}
01 2000
{'01': 1978, '00': 22}
00 2000
{'01': 19, '00': 1981}
10 2000
{'11': 20, '10': 1980}
01 2000
{'00': 26, '01': 1974}
01 2000
{'00': 25, '01': 1975}

Depth = 4
Number of samples = 20
01 2000
{'00': 39, '01': 1961}
10 2000
{'11': 40, '10': 1960}
00 2000
{'01': 37, '00': 1963}
11 2000
{'10': 47, '11': 1953}
10 2000
{'11': 41, '10': 1959}
00 2000
{'01': 43, '00': 1957}
01 2000
{'00': 34, '01': 1966}
10 2000
{'11': 30, '10': 1970}
00 2000
{'01': 48, '00': 1952}
11 2000
{'10': 2

10 2000
{'11': 39, '10': 1961}
00 2000
{'01': 36, '00': 1964}
11 2000
{'10': 35, '11': 1965}
01 2000
{'00': 39, '01': 1961}
11 2000
{'10': 43, '11': 1957}
10 2000
{'11': 42, '10': 1958}
11 2000
{'10': 49, '11': 1951}
10 2000
{'11': 33, '10': 1967}
11 2000
{'10': 47, '11': 1953}
11 2000
{'10': 48, '11': 1952}
10 2000
{'11': 36, '10': 1964}

Depth = 16
Number of samples = 20
00 2000
{'01': 56, '00': 1944}
10 2000
{'11': 74, '10': 1926}
11 2000
{'10': 87, '11': 1913}
11 2000
{'10': 78, '11': 1922}
11 2000
{'10': 61, '11': 1939}
01 2000
{'00': 83, '01': 1917}
10 2000
{'11': 79, '10': 1921}
01 2000
{'00': 71, '01': 1929}
00 2000
{'01': 67, '00': 1933}
11 2000
{'10': 75, '11': 1925}
01 2000
{'00': 65, '01': 1935}
11 2000
{'10': 77, '11': 1923}
11 2000
{'10': 75, '11': 1925}
01 2000
{'00': 76, '01': 1924}
01 2000
{'00': 94, '01': 1906}
01 2000
{'00': 75, '01': 1925}
00 2000
{'01': 70, '00': 1930}
10 2000
{'11': 67, '10': 1933}
11 2000
{'10': 80, '11': 1920}
01 2000
{'00': 77, '01': 1923}

Dep

00 2000
{'11': 22, '10': 41, '00': 1937}
00 2000
{'10': 33, '11': 21, '00': 1946}
10 2000
{'01': 26, '00': 40, '10': 1934}
10 2000
{'01': 19, '00': 42, '10': 1939}
10 2000
{'01': 15, '11': 1, '00': 34, '10': 1950}
01 2000
{'10': 22, '11': 41, '01': 1937}
00 2000
{'11': 21, '10': 32, '00': 1947}
11 2000
{'01': 36, '00': 24, '11': 1940}
00 2000
{'01': 1, '11': 21, '10': 35, '00': 1943}
10 2000
{'01': 17, '00': 35, '10': 1948}
01 2000
{'00': 1, '10': 22, '11': 40, '01': 1937}

Depth = 8
Number of samples = 20
01 2000
{'00': 4, '10': 39, '11': 70, '01': 1887}
11 2000
{'01': 78, '10': 2, '00': 47, '11': 1873}
01 2000
{'00': 4, '10': 32, '11': 85, '01': 1879}
10 2000
{'01': 26, '11': 1, '00': 81, '10': 1892}
11 2000
{'01': 77, '00': 37, '10': 2, '11': 1884}
11 2000
{'01': 82, '00': 33, '10': 1, '11': 1884}
01 2000
{'00': 3, '10': 26, '11': 85, '01': 1886}
01 2000
{'00': 1, '10': 28, '11': 67, '01': 1904}
11 2000
{'01': 83, '10': 1, '00': 49, '11': 1867}
11 2000
{'01': 77, '00': 35, '10': 2, 

11 2000
{'01': 321, '00': 19, '10': 149, '11': 1511}
01 2000
{'10': 22, '00': 115, '11': 369, '01': 1494}
10 2000
{'01': 24, '11': 118, '00': 364, '10': 1494}
11 2000
{'01': 336, '00': 33, '10': 105, '11': 1526}
01 2000
{'01': 1490, '10': 21, '00': 119, '11': 370}
01 2000
{'00': 122, '11': 370, '10': 33, '01': 1475}
10 2000
{'01': 24, '00': 325, '10': 1514, '11': 137}
10 2000
{'01': 21, '11': 128, '00': 351, '10': 1500}
00 2000
{'01': 130, '11': 29, '00': 1479, '10': 362}

pauli label: ZX

Depth = 2
Number of samples = 20
00 2000
{'01': 13, '11': 12, '10': 24, '00': 1951}
11 2000
{'01': 17, '00': 9, '10': 14, '11': 1960}
01 2000
{'11': 19, '00': 9, '10': 11, '01': 1961}
11 2000
{'01': 21, '00': 9, '10': 14, '11': 1956}
11 2000
{'01': 23, '10': 7, '00': 8, '11': 1962}
01 2000
{'00': 16, '10': 12, '11': 23, '01': 1949}
10 2000
{'01': 12, '00': 23, '11': 10, '10': 1955}
10 2000
{'01': 11, '00': 18, '11': 15, '10': 1956}
10 2000
{'01': 13, '11': 9, '00': 21, '10': 1957}
10 2000
{'01': 9, '

01 2000
{'00': 2, '11': 76, '10': 33, '01': 1889}
10 2000
{'01': 38, '11': 1, '00': 90, '10': 1871}
01 2000
{'00': 1, '10': 35, '11': 81, '01': 1883}
11 2000
{'01': 75, '10': 3, '00': 41, '11': 1881}
01 2000
{'00': 2, '11': 72, '10': 39, '01': 1887}
00 2000
{'11': 29, '10': 72, '00': 1899}
10 2000
{'01': 29, '11': 2, '00': 76, '10': 1893}
01 2000
{'00': 3, '11': 84, '10': 45, '01': 1868}
00 2000
{'01': 1, '11': 31, '10': 78, '00': 1890}

Depth = 16
Number of samples = 20
01 2000
{'00': 7, '10': 61, '11': 129, '01': 1803}
01 2000
{'00': 4, '10': 67, '11': 134, '01': 1795}
10 2000
{'11': 9, '00': 132, '10': 1791, '01': 68}
00 2000
{'01': 4, '10': 139, '11': 63, '00': 1794}
01 2000
{'00': 4, '10': 58, '11': 130, '01': 1808}
10 2000
{'01': 63, '11': 4, '00': 133, '10': 1800}
11 2000
{'01': 143, '10': 5, '11': 1782, '00': 70}
11 2000
{'01': 174, '10': 6, '00': 68, '11': 1752}
01 2000
{'00': 5, '10': 65, '11': 138, '01': 1792}
10 2000
{'01': 74, '11': 9, '10': 1770, '00': 147}
01 2000
{'00':

In [64]:
print (all_circuits_explicit["XX"][2])

['RY+22:RY+32:AZ2+1.0000:RY+31:FTXA230.2500+:RY+21:FTXA230.2500+:RY+21:FTXA230.2500+:RY+21:RX+31:FTXA230.2500+:RY+21:RX+31:FTXA230.2500+:AZ2+1.0000:RY+31:FTXA230.2500+:RY+31:FTXA230.2500+:AZ2+1.0000:RX+31:FTXA230.2500+:AZ2+1.0000:AZ3+1.0000:RY-22:RY-32', 'RY+22:RY+32:RX+21:RX+31:FTXA230.2500+:RX+31:FTXA230.2500+:RY+31:FTXA230.2500+:AZ2+1.0000:RX+31:FTXA230.2500+:RX+21:RX+31:FTXA230.2500+:AZ3+1.0000:FTXA230.2500+:RY+21:RY+31:FTXA230.2500+:AZ2+1.0000:FTXA230.2500+:AZ2+1.0000:RX+31:RY-22:RY-32', 'RY+22:RY+32:RX+21:AZ3+1.0000:FTXA230.2500+:RY+31:FTXA230.2500+:AZ2+1.0000:FTXA230.2500+:AZ2+1.0000:RY+31:FTXA230.2500+:RX+21:AZ3+1.0000:FTXA230.2500+:RX+21:AZ3+1.0000:FTXA230.2500+:RY+21:RY+31:FTXA230.2500+:AZ3+1.0000:FTXA230.2500+:AZ3+1.0000:RY-22:RY-32', 'RY+22:RY+32:RX+21:AZ3+1.0000:FTXA230.2500+:RY+21:RY+31:FTXA230.2500+:RX+21:RX+31:FTXA230.2500+:RX+21:AZ3+1.0000:FTXA230.2500+:RX+21:RY+31:FTXA230.2500+:AZ2+1.0000:FTXA230.2500+:AZ2+1.0000:AZ3+1.0000:FTXA230.2500+:RX+31:FTXA230.2500+:AZ2+1.0000

In [65]:
print (cb_data)

{'XX': {'parameters': {'n': 2, 'shots': 2000, 'Lrange': [2, 4, 8, 16, 32], 'C': 20}, 'result': [{'L': 2, 'circuit': <bound method QuantumCircuit.qasm of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc477085730>>, 'clifford': {'stabilizer': ['-IZ', '-ZI'], 'destabilizer': ['+IX', '-XI']}, 'clifford_layer': 'XX', 'pauli': 'XX', 'counts': {'00': 0, '10': 2000, '01': 0, '11': 0}}, {'L': 2, 'circuit': <bound method QuantumCircuit.qasm of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc482f1a340>>, 'clifford': {'stabilizer': ['+IZ', '+ZI'], 'destabilizer': ['-IX', '+XI']}, 'clifford_layer': 'XX', 'pauli': 'XX', 'counts': {'00': 0, '10': 0, '01': 2000, '11': 0}}, {'L': 2, 'circuit': <bound method QuantumCircuit.qasm of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc480973d00>>, 'clifford': {'stabilizer': ['+IZ', '-ZI'], 'destabilizer': ['-IX', '+XI']}, 'clifford_layer': 'XX', 'pauli': 'XX', 'counts': {'00': 0, '10': 0, '01': 2000, '11': 0}}, {'L': 2, '

In [66]:
pauli_sample_list = [''.join(s) for s in itertools.product(['X','Y','Z'], repeat = n)]
cb_data["n"] = n
cb_data["pauli_sample_list"] = pauli_sample_list 

filename = 'sim_cb_cnot_2023sep'
token = ''.join(random.choice([str(j) for j in range(10)]) for i in range(10))
filename += '_' + token
with open('/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation/data/CB_test/' + filename, 'wb') as outfile:
    pickle.dump(cb_data, outfile)

In [67]:
print (filename)

sim_cb_cnot_2023sep_9718455994


In [70]:
# print (all_circuits_explicit[0])
for pauli_sample in pauli_sample_list:
    for i in range(5):
        cricfname=pauli_sample+"_d="+str(2**(i+1))+".txt"
        with open("/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation/circuits/CB/XX 10102023/"+cricfname, "w") as text_file:
            for line in all_circuits_explicit[pauli_sample][i]:
                text_file.write(line+"\n")

# with open("/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation/circuits/XX23541_noSE_explicitCNOT_FT_R0627_01+10/d=4.txt", "w") as text_file:
#     for line in all_circuits_explicit[1]:
#         text_file.write(line+"\n")
# with open("/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation/circuits/XX23541_noSE_explicitCNOT_FT_R0627_01+10/d=8.txt", "w") as text_file:
#     for line in all_circuits_explicit[2]:
#         text_file.write(line+"\n")
# with open("/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation/circuits/XX23541_noSE_explicitCNOT_FT_R0627_01+10/d=16.txt", "w") as text_file:
#     for line in all_circuits_explicit[3]:
#         text_file.write(line+"\n")
# with open("/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation/circuits/XX23541_noSE_explicitCNOT_FT_R0627_01+10/d=32.txt", "w") as text_file:
#     for line in all_circuits_explicit[4]:
#         text_file.write(line+"\n")

In [2]:
import sys
sys.path.append(r"/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation/")
import CB_process_modified
import pdb
'''Use these if read from file'''
# token = "8799418287"
# filename = "sim_cb_cnot_2023sep_9167558962"
with open('/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation/data/CB_test/' + filename, 'rb') as infile:
    cb_data = pickle.load(infile)

data=cb_data

n = data["n"]
pauli_sample_list = data["pauli_sample_list"]
'''Specify a set of Pauli you want to estimate'''
pauli_request_list = [''.join(s) for s in itertools.product(['I','X','Y','Z'], repeat = n)] #full

fidelity_list = {} 
stdev_list = {}

for pauli_sample in pauli_sample_list:
    cb_data_temp = data[pauli_sample]

    # n = cb_data["parameters"]['n']
    # n_total = cb_data["parameters"]['n_total'] 
    shots = cb_data_temp ["parameters"]['shots'] 
    Lrange = cb_data_temp ["parameters"]['Lrange']
    C = cb_data_temp ["parameters"]['C'] 
#     eps_readout = cb_data_temp ["parameters"]['eps_readout'] 
#     repeat = cb_data_temp ["parameters"]['repeat']
#     pdb.set_trace()
    cb_result = CB_process_modified.process_CB(n, C, shots, 1, Lrange, cb_data_temp , pauli_sample = pauli_sample, periodic=True,use_density_matrix=False, intercept_cb=False)
    raw_fidelity_list = cb_result["fidelity_list"]
    
    new_sub_label = []

    for sub_label in raw_fidelity_list.keys():
        if sub_label in fidelity_list:
            continue # wasteful!
        elif(sub_label == 'I'*n):
            fidelity_list[sub_label] = 1.0
            stdev_list[sub_label] = 0.0
        else:
            alpha, alpha_err = CB_process_modified.fit_CB(Lrange, raw_fidelity_list[sub_label])
            fidelity_list[sub_label] = alpha
            stdev_list[sub_label] = alpha_err
            new_sub_label.append(sub_label)

    print("CB setting: ",pauli_sample[::-1]," Pauli fidelities calculated: ", [sub_label[::-1] for sub_label in new_sub_label])


# print(fidelity_list)

# print(stdev_list)
print("Parameters: n = %d, C = %d, " % (n,C), "L = ", str(Lrange))



# Average fidelity
print("Total error = ", 1-np.mean(list(fidelity_list.values())))

print("Label / Pauli infidelity / Standard deviation")
for pauli_label in pauli_request_list:
    print(pauli_label[::-1], 1-fidelity_list[pauli_label], stdev_list[pauli_label])

NameError: name 'cb_data' is not defined

In [48]:
print (raw_fidelity_list)

{'II': {2: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 4: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 8: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 16: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 32: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, 'IZ': {2: [0.978, 0.977, 0.967, 0.973, 0.982, 0.98, 0.987, 0.976, 0.98, 0.986, 0.977, 0.98, 0.983, 0.982, 0.975, 0.986, 0.987, 0.979, 0.983, 0.986], 4: [0.962, 0.965, 0.957, 0.937, 0.967, 0.96, 0.964, 0.951, 0.968, 0.964, 0.961, 0.955, 0.963, 0.964, 0.959, 0.965, 0.957, 0.951, 0.963, 0.955], 8: [0.921, 0.919, 0.921, 0.925, 0.903, 0.925, 0.936, 0.91, 0.913, 0.933, 0.925, 0.918, 0.92, 0.932, 0.925, 0.914, 0.931, 0.93, 0.93, 0.924], 16: [0.855, 0.853, 0.84, 0.86,

In [49]:
# raw_fidelity_list = eab_result["fidelity_list"]


# # sys.exit(0)

# fidelity_list = {}
# stdev_list = {}

# for pauli_label in pauli_request_list:
#     if(pauli_label == 'I'*nqubit):
#         fidelity_list[pauli_label] = 1.0
#         stdev_list[pauli_label] = 0.0
#     else:
#         alpha, alpha_err = EAB_process_modified.fit_EAB(depth, raw_fidelity_list[pauli_label])
#         fidelity_list[pauli_label] = alpha
#         stdev_list[pauli_label] = alpha_err

# # print(fidelity_list)

# # print(stdev_list)
# print("Parameters: n = %d, C = %d, " % (nqubit,C), "L = ", str(depth))
# use_density_matrix=False
# use_readout_error =False
# shots=2000
# if use_density_matrix:
#     print("Density matrix based simulation")
# else:
#     print("Measurement based simulation, shots = %d" % shots)

# if use_readout_error is True:
#     print("Measurement bitflip rate = %f" % eps_readout)
# else:
#     print("No readout error")

# # Average fidelity
# print("Total error = ", 1-np.mean(list(fidelity_list.values())))

# print("Label / Pauli infidelity / Standard deviation")
# for pauli_label in pauli_request_list:
#     print(pauli_label[::-1], 1-fidelity_list[pauli_label], stdev_list[pauli_label])
# # print('Effective noise rate = ' + str(1-np.average(list(fidelity_list.values()))))



In [50]:
error_list = fidelity_to_error(fidelity_list,n)
print("Label / Pauli error rates")
for pauli_label in pauli_request_list:
    print(pauli_label[::-1], error_list[pauli_label])

Label / Pauli error rates
II 0.9900090212290751
XI 0.002505046646998091
YI -9.364653391297217e-06
ZI 9.364653391297217e-06
IX 0.004995370711939742
XX 0.0024905614119871064
YX 5.7077549728223875e-05
ZX -5.7077549728223875e-05
IY 3.8624901991864136e-05
XY 1.7033412224448652e-07
YY 9.202053893750395e-06
ZY -9.202053893750395e-06
IZ -3.8624901991864136e-05
XZ -1.7033412224448652e-07
YZ -9.202053893750395e-06
ZZ 9.202053893750395e-06


In [37]:
# True value for this specific noise model
eps=0
def f_true(P):
    ans = 0.0
    for Pi in P:
        if Pi == 'Z' or Pi == 'Y':
            ans += 2*eps
    return ans
def p_true(P):
    if P == "XI" or P == "IX":
        return eps
    else:
        return 0.0

print("Label / True infidelity/ True Pauli error")
for P in pauli_request_list:
    print(P[::-1]," ",f_true(P)," ",p_true(P))

Label / True infidelity/ True Pauli error
II   0.0   0.0
XI   0.0   0
YI   0.0   0.0
ZI   0.0   0.0
IX   0.0   0
XX   0.0   0.0
YX   0.0   0.0
ZX   0.0   0.0
IY   0.0   0.0
XY   0.0   0.0
YY   0.0   0.0
ZY   0.0   0.0
IZ   0.0   0.0
XZ   0.0   0.0
YZ   0.0   0.0
ZZ   0.0   0.0


In [ ]:
print (raw_fidelity_list)

In [41]:
label="-XY"
print (label[-2:])

XY
